In [1]:
import requests, uuid, json, os
from iso639 import languages
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

## Importation des jeux de données


In [2]:
# Fichier contenant les codes iso
df_iso = pd.read_fwf('y_test.txt', header=None)

# Fichier contenant les paragraphes
df_text = pd.read_fwf('x_test.txt', header=None)
df_text = df_text[0]

In [3]:
def find_text(iso_alpha2: str, quantity: int, df_iso=df_iso, df_text=df_text):
    """
    Lis les fichiers 'y_test.txt' et 'x_test.txt'.
    Permet de retourner une liste de paragraphes selon l'iso renseigné.
    Le paramètre 'iso_alpha2' doit être de 2 caractères, exemple : 'fr'.
    Le paramètre 'quantity' permet de choisir le nombre de paragraphes que
    l'on veut (500 max).
    Retourne la liste de paragraphe.
    Exemple d'utilisation :
    >>> textes_italien = find_text('it', 5)
    >>> textes_italien
    >>> ['Spesso usata quale motto o incitazione, ha il significato...', ...]
    """
    # Conversion du code iso alpha2 vers alpha3
    code_alpha3 = languages.get(alpha2=iso_alpha2).part3
    
    # Sélection des lignes selon l'iso renseigné
    df_iso = df_iso[df_iso == code_alpha3].dropna()
    
    # Récupération des index
    lines = df_iso.index.tolist()
    lines = lines[:quantity]

    paragraphes = []
    # Utilisation des index pour récupérer les paragraphes correspondant
    for i in lines:
        paragraphes.append(df_text[i])

    return paragraphes

## Les identifiants et clés de connection ne sont pas visibles dans le script

In [4]:
subscription_key = os.getenv("SUBSCRIPTION_KEY")
endpoint = os.getenv("ENDPOINT")
location = os.getenv("LOCATION")

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'to': ['fr']
}
constructed_url = endpoint + path

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

In [5]:
def detectlang(text='Hello World!'):
    """
    Permet de détecter une langue selon le texte entré en paramètre.
    On obtient en sortie la langue detectée et son score
    """
    body = [{
        'text': str(text)
    }]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    response = json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': '))

    detectedLanguage = response.find('"language": "')
    detectedLanguage = response[detectedLanguage + len('"language": "'):detectedLanguage + len('"language": "') + 2]

    score = response.find('"score": ')
    score = response[score + len('"score": '):score + len('"score": ') + 3]

    return detectedLanguage, score

### Les 5 langues les plus parlées dans le monde sont :
- L'anglais
- Le chinois
- L'hindi
- L'espagnol
- L'arabe

In [6]:
top_lang = ['en', 'zh', 'hi', 'es', 'ar']

In [7]:
# Dictionnaire pour stocker les résultats
dico = {
    'Texte': [],
    'Langue': [],
    'LangueDetectée': [],
    'Score': []
    }

### On vérifie ici sur plusieurs paragraphes en langues différentes si l'api de Microsoft retourne bien le résultat voulu
On peut modifier le nombre de paragraphes en modifiant le deuxième paramètre de la fonction 'find_text'.

In [8]:
for langue in top_lang:
    paragraphes = find_text(langue, 10)
    for paragraphe in paragraphes:

        detectedLanguage, score = detectlang(paragraphe)

        dico['Texte'].append(paragraphe)
        dico['Langue'].append(langue)
        dico['LangueDetectée'].append(detectedLanguage)
        dico['Score'].append(float(score))

In [9]:
resultats = pd.DataFrame(dico)
resultats

,Texte,Langue,LangueDetectée,Score
0,Anton (or Antonius) Maria Schyrleus (also Schy...,en,en,0.9
1,Zvi Zamir (Hebrew: צבי זמיר‎) born Zvicka Zarz...,en,en,0.9
2,Texas A&M hired Kennedy in May 2011 after Mark...,en,en,1.0
3,"Philip Johnson architectural drawings, 1943-19...",en,en,1.0
4,"AVCO Embassy Pictures, the film's financial ba...",en,en,1.0
5,"Ilex mucronata, formerly the type species of N...",en,en,0.9
6,"Orner, along with actress Cate Blanchett, was ...",en,en,1.0
7,"The Berliner Joyce design, a conventional bipl...",en,en,1.0
8,Inspired by his tempestuous relationships with...,en,en,1.0
9,All 14 tracks were written and produced by Woo...,en,en,1.0


## Résultats obtenus
Lorsqu'on calcule la moyenne de score par langue, on obtient un score >= 0.96 pour 10 paragraphes par langue.

In [10]:
scoreMoyenParLangue = resultats.groupby('Langue')['Score'].mean()
scoreMoyenParLangue

Langue
ar    0.99
en    0.97
es    0.96
hi    0.96
zh    0.97
Name: Score, dtype: float64

## La requête écrite permet de récupérer la prédiction du modèle
Il suffit d'éxecuter la cellule ci-dessous et d'entrer le texte voulu.

In [11]:
value = input("Entrez un texte : ")
returnLang, score = detectlang(str(value))
print(f'Langue detecté : {returnLang}')
print(f'Score : {score}')

Entrez un texte : Bonjour tout le monde !
Langue detecté : fr
Score : 1.0
